# 02 - Feature Engineering + Machine Learning

**Projet :** Electio-Analytics - Prediction Presidentielles 2027  
**Schema :** v3.0 (534 communes Gironde)  

---

## Strategie ML (ADR-002)

- **Unite d'analyse** : Commune (534)
- **Train** : Features 2017 T1 -> Target 2022 T1 (% voix par candidat)
- **Predict** : Features 2022 T1 -> Prediction 2027 T1
- **7 candidats communs** : 1 modele Random Forest par candidat
- **Baseline** : Linear Regression
- **Validation** : K-Fold CV (k=5)
- **Objectifs** : R2 > 0.65, MAE < 3 pts, RMSE < 4 pts

### Features par commune

| Feature | Source |
|---------|--------|
| `pct_voix_{candidat}_prev` | resultat_candidat (election precedente) |
| `pct_voix_autres_prev` | somme des autres candidats |
| `taux_participation_prev` | resultat_participation |
| `taux_abstention_prev` | resultat_participation |
| `population` | commune |
| `log_population` | log(population) |
| `securite_*` (5 cols) | indicateur (0 si pas Bordeaux) |

**Note technique :** Les `id_territoire` des resultats electoraux utilisent le format `'33' + code_insee` (7 chars),
tandis que `commune.id_commune` = code INSEE brut (5 chars). On normalise via `id_territoire[2:]`.

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)

import sys
sys.path.insert(0, '..')

from sqlalchemy import func
from src.database.config import get_engine, get_session
from src.database.models import (
    Commune, Election, Candidat,
    ResultatParticipation, ResultatCandidat,
    TypeIndicateur, Indicateur, Prediction
)

engine = get_engine()
session = get_session()
print("Connexion DB OK")

Connexion DB OK


In [ ]:
# Charger donnees brutes via ORM

# Communes Gironde
query_communes = session.query(
    Commune.id_commune, Commune.nom_commune, Commune.population, Commune.superficie_km2
).filter(Commune.id_departement == '33')
df_communes = pd.read_sql(query_communes.statement, session.bind)

# Participation (JOIN Election)
query_participation = session.query(
    ResultatParticipation.id_territoire,
    ResultatParticipation.tour,
    ResultatParticipation.nombre_inscrits,
    ResultatParticipation.nombre_votants,
    ResultatParticipation.nombre_exprimes,
    ResultatParticipation.pourcentage_votants,
    ResultatParticipation.pourcentage_abstentions,
    Election.annee
).join(
    Election, ResultatParticipation.id_election == Election.id_election
).filter(
    ResultatParticipation.type_territoire == 'COMMUNE'
)
df_participation = pd.read_sql(query_participation.statement, session.bind)
# Normaliser id_territoire -> code INSEE
df_participation['commune_id'] = df_participation['id_territoire'].str[2:]

# Resultats candidats (JOIN Election + Candidat)
query_resultats = session.query(
    ResultatCandidat.id_territoire,
    ResultatCandidat.tour,
    ResultatCandidat.nombre_voix,
    ResultatCandidat.pourcentage_voix_exprimes,
    Election.annee,
    Candidat.nom,
    Candidat.prenom,
    Candidat.id_candidat
).join(
    Election, ResultatCandidat.id_election == Election.id_election
).join(
    Candidat, ResultatCandidat.id_candidat == Candidat.id_candidat
).filter(
    ResultatCandidat.type_territoire == 'COMMUNE'
)
df_resultats = pd.read_sql(query_resultats.statement, session.bind)
# Normaliser id_territoire -> code INSEE
df_resultats['commune_id'] = df_resultats['id_territoire'].str[2:]
df_resultats['candidat_nom'] = df_resultats['prenom'] + ' ' + df_resultats['nom']
# NULL pourcentage = 0 voix = 0%
df_resultats['pct'] = df_resultats['pourcentage_voix_exprimes'].fillna(0).astype(float)

# Indicateurs securite Bordeaux (JOIN TypeIndicateur)
query_indicateurs = session.query(
    Indicateur.annee,
    Indicateur.valeur_numerique,
    Indicateur.id_territoire,
    TypeIndicateur.code_type
).join(
    TypeIndicateur, Indicateur.id_type == TypeIndicateur.id_type
).filter(
    Indicateur.id_territoire == '33063',
    Indicateur.type_territoire == 'COMMUNE'
)
df_indicateurs = pd.read_sql(query_indicateurs.statement, session.bind)

print(f"Communes      : {len(df_communes)}")
print(f"Participation : {len(df_participation)}")
print(f"Resultats     : {len(df_resultats)}")
print(f"Indicateurs   : {len(df_indicateurs)}")

In [3]:
# Identifier 7 candidats communs 2017-2022 (T1 uniquement)
t1 = df_resultats[df_resultats['tour'] == 1]
cands_2017 = set(t1[t1['annee'] == 2017]['candidat_nom'].unique())
cands_2022 = set(t1[t1['annee'] == 2022]['candidat_nom'].unique())
candidats_communs = sorted(cands_2017 & cands_2022)

print(f"{len(candidats_communs)} candidats communs :")
for c in candidats_communs:
    print(f"  - {c}")

7 candidats communs :
  - Emmanuel MACRON
  - Jean LASSALLE
  - Jean-Luc MÉLENCHON
  - Marine LE PEN
  - Nathalie ARTHAUD
  - Nicolas DUPONT-AIGNAN
  - Philippe POUTOU


In [4]:
# Construire matrice features

def build_features(df_resultats, df_participation, df_communes, df_indicateurs,
                   annee, candidats_communs, annee_indicateur=None):
    """
    Construit la matrice de features pour une annee donnee.
    Utilise commune_id (code INSEE 5 chars) comme index.
    """
    if annee_indicateur is None:
        annee_indicateur = annee

    # 1. Pivoter % voix T1 par candidat
    t1 = df_resultats[(df_resultats['annee'] == annee) & (df_resultats['tour'] == 1)].copy()
    t1_communs = t1[t1['candidat_nom'].isin(candidats_communs)]

    pivot_voix = t1_communs.pivot_table(
        index='commune_id', columns='candidat_nom', values='pct', aggfunc='first'
    )
    pivot_voix.columns = [f'pct_{c.split()[-1].lower()}_prev' for c in pivot_voix.columns]

    # Somme des "autres" candidats (hors 7 communs)
    t1_autres = t1[~t1['candidat_nom'].isin(candidats_communs)]
    autres = t1_autres.groupby('commune_id')['pct'].sum().rename('pct_autres_prev')

    # 2. Participation T1
    part = df_participation[(df_participation['annee'] == annee) & (df_participation['tour'] == 1)].copy()
    part = part.set_index('commune_id')[['pourcentage_votants', 'pourcentage_abstentions']]
    part.columns = ['taux_participation_prev', 'taux_abstention_prev']
    part = part.astype(float)

    # 3. Population
    pop = df_communes.set_index('id_commune')[['population']].copy()
    pop['log_population'] = np.log1p(pop['population'].fillna(0))

    # 4. Indicateurs securite (pivot par code_type, 0 si pas Bordeaux)
    indic = df_indicateurs[df_indicateurs['annee'] == annee_indicateur].copy()
    if len(indic) > 0:
        indic_pivot = indic.pivot_table(
            index='id_territoire', columns='code_type',
            values='valeur_numerique', aggfunc='first'
        ).astype(float)
        indic_pivot.columns = [f'securite_{c.lower()}' for c in indic_pivot.columns]
    else:
        indic_pivot = pd.DataFrame()

    # Assembler (tout indexe sur code INSEE 5 chars)
    features = pivot_voix.join(autres, how='left').join(part, how='left').join(pop, how='left')
    if len(indic_pivot) > 0:
        features = features.join(indic_pivot, how='left')

    # Remplir NaN securite par 0 (seul Bordeaux a des indicateurs)
    securite_cols = [c for c in features.columns if c.startswith('securite_')]
    features[securite_cols] = features[securite_cols].fillna(0)

    # Remplir les rares NaN restants
    features = features.fillna(0)

    return features


# Features 2017 (pour train) - indicateurs 2017
X_2017 = build_features(df_resultats, df_participation, df_communes, df_indicateurs,
                        annee=2017, candidats_communs=candidats_communs, annee_indicateur=2017)

# Features 2022 (pour predict 2027) - indicateurs 2022
X_2022 = build_features(df_resultats, df_participation, df_communes, df_indicateurs,
                        annee=2022, candidats_communs=candidats_communs, annee_indicateur=2022)

print(f"X_2017 : {X_2017.shape} ({X_2017.shape[0]} communes x {X_2017.shape[1]} features)")
print(f"X_2022 : {X_2022.shape}")
print(f"\nFeatures : {list(X_2017.columns)}")

X_2017 : (538, 17) (538 communes x 17 features)
X_2022 : (535, 17)

Features : ['pct_macron_prev', 'pct_lassalle_prev', 'pct_mélenchon_prev', 'pct_pen_prev', 'pct_arthaud_prev', 'pct_dupont-aignan_prev', 'pct_poutou_prev', 'pct_autres_prev', 'taux_participation_prev', 'taux_abstention_prev', 'population', 'log_population', 'securite_atteintes_aux_biens', 'securite_atteintes_aux_personnes', 'securite_criminalite_totale', 'securite_vols_avec_violence', 'securite_vols_sans_violence']


In [ ]:
# Construire targets : 2022 T1 % voix exprimes par candidat (pour train)
t1_2022 = df_resultats[
    (df_resultats['annee'] == 2022) & (df_resultats['tour'] == 1) &
    (df_resultats['candidat_nom'].isin(candidats_communs))
]

targets = {}
for candidat in candidats_communs:
    y = t1_2022[t1_2022['candidat_nom'] == candidat].set_index('commune_id')['pct']
    targets[candidat] = y

# Aligner sur les communes presentes dans X_2017
# Utiliser l'union de toutes les communes presentes dans au moins un target
all_target_communes = set()
for y in targets.values():
    all_target_communes.update(y.index)
communes_communes = X_2017.index.intersection(list(all_target_communes))
X_train = X_2017.loc[communes_communes]
X_predict = X_2022.loc[X_2022.index.isin(communes_communes)]

# Reindex targets pour garantir 0 pour les communes sans resultat
for candidat in candidats_communs:
    targets[candidat] = targets[candidat].reindex(communes_communes, fill_value=0.0)

print(f"Communes alignees : {len(communes_communes)}")
print(f"X_train : {X_train.shape}")
print(f"X_predict : {X_predict.shape}")
print(f"\nDistribution targets (2022 T1) :")
for c in candidats_communs:
    y = targets[c]
    print(f"  {c:30s} : mean={y.mean():.2f}%, std={y.std():.2f}, min={y.min():.2f}, max={y.max():.2f}")

In [ ]:
# Correlation + distributions
import os
os.makedirs('../docs/figures/ml', exist_ok=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Heatmap correlation features
corr = X_train.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            ax=ax1, square=True, linewidths=0.5, cbar_kws={'shrink': 0.8},
            annot_kws={'size': 7})
ax1.set_title('Correlation features (X_train 2017)', fontsize=12)

# Distribution des targets
target_means = {c.split()[-1]: targets[c].loc[communes_communes].mean() for c in candidats_communs}
ax2.barh(list(target_means.keys()), list(target_means.values()), color='coral')
ax2.set_xlabel('% voix exprimes moyen (2022 T1)')
ax2.set_title('Targets : % moyen par candidat')
for i, v in enumerate(target_means.values()):
    ax2.text(v + 0.3, i, f'{v:.1f}%', va='center')

plt.tight_layout()
plt.savefig('../docs/figures/ml/correlation_features.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Baseline Linear Regression - Cross-validation 5-fold par candidat
kf = KFold(n_splits=5, shuffle=True, random_state=42)

results_lr = []
models_lr = {}

for candidat in candidats_communs:
    y = targets[candidat].values.astype(float)
    X = X_train.values

    lr = LinearRegression()
    scores_r2 = cross_val_score(lr, X, y, cv=kf, scoring='r2')
    scores_mae = -cross_val_score(lr, X, y, cv=kf, scoring='neg_mean_absolute_error')
    scores_rmse = np.sqrt(-cross_val_score(lr, X, y, cv=kf, scoring='neg_mean_squared_error'))

    lr.fit(X, y)
    models_lr[candidat] = lr

    results_lr.append({
        'candidat': candidat.split()[-1],
        'R2_mean': scores_r2.mean(),
        'R2_std': scores_r2.std(),
        'MAE_mean': scores_mae.mean(),
        'RMSE_mean': scores_rmse.mean()
    })

df_lr = pd.DataFrame(results_lr)
print("=== Baseline : Linear Regression (5-Fold CV) ===")
print(df_lr.to_string(index=False))

In [ ]:
# Random Forest (modele principal) - Cross-validation 5-fold par candidat
results_rf = []
models_rf = {}

for candidat in candidats_communs:
    y = targets[candidat].values.astype(float)
    X = X_train.values

    rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
    scores_r2 = cross_val_score(rf, X, y, cv=kf, scoring='r2')
    scores_mae = -cross_val_score(rf, X, y, cv=kf, scoring='neg_mean_absolute_error')
    scores_rmse = np.sqrt(-cross_val_score(rf, X, y, cv=kf, scoring='neg_mean_squared_error'))

    rf.fit(X, y)
    models_rf[candidat] = rf

    results_rf.append({
        'candidat': candidat.split()[-1],
        'R2_mean': scores_r2.mean(),
        'R2_std': scores_r2.std(),
        'MAE_mean': scores_mae.mean(),
        'RMSE_mean': scores_rmse.mean()
    })

df_rf = pd.DataFrame(results_rf)
print("=== Random Forest (5-Fold CV) ===")
print(df_rf.to_string(index=False))

In [ ]:
# Comparaison LR vs RF
df_compare = df_lr[['candidat', 'R2_mean', 'MAE_mean', 'RMSE_mean']].rename(
    columns={'R2_mean': 'LR_R2', 'MAE_mean': 'LR_MAE', 'RMSE_mean': 'LR_RMSE'}
).merge(
    df_rf[['candidat', 'R2_mean', 'MAE_mean', 'RMSE_mean']].rename(
        columns={'R2_mean': 'RF_R2', 'MAE_mean': 'RF_MAE', 'RMSE_mean': 'RF_RMSE'}
    ),
    on='candidat'
)

print("=== Comparaison Linear Regression vs Random Forest ===")
print(df_compare.to_string(index=False))

# Barplot comparatif R2
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(df_compare))
w = 0.35
ax.bar(x - w/2, df_compare['LR_R2'], w, label='Linear Regression', color='steelblue')
ax.bar(x + w/2, df_compare['RF_R2'], w, label='Random Forest', color='coral')
ax.set_xticks(x)
ax.set_xticklabels(df_compare['candidat'], rotation=30, ha='right')
ax.set_ylabel('R2 (cross-validation 5-fold)')
ax.set_title('Comparaison R2 : Linear Regression vs Random Forest')
ax.axhline(0.65, color='green', ls='--', alpha=0.5, label='Objectif R2=0.65')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../docs/figures/ml/comparaison_lr_rf.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Feature importance - top features pour Macron, Le Pen, Melenchon
focus_candidats = [c for c in candidats_communs if any(n in c.upper() for n in ['MACRON', 'LE PEN', 'MELENCHON', 'MÉLENCHON'])]
# Fallback : prendre les 3 premiers si le filtre ne marche pas
if len(focus_candidats) < 3:
    focus_candidats = candidats_communs[:3]

fig, axes = plt.subplots(1, len(focus_candidats), figsize=(6 * len(focus_candidats), 6))
if len(focus_candidats) == 1:
    axes = [axes]

feature_names = list(X_train.columns)

for ax, candidat in zip(axes, focus_candidats):
    rf = models_rf[candidat]
    importances = rf.feature_importances_
    idx_sorted = np.argsort(importances)
    ax.barh(range(len(importances)), importances[idx_sorted], color='coral')
    ax.set_yticks(range(len(importances)))
    ax.set_yticklabels([feature_names[i] for i in idx_sorted], fontsize=8)
    ax.set_xlabel('Importance')
    ax.set_title(f'{candidat.split()[-1]}')

plt.suptitle('Feature Importance (Random Forest)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../docs/figures/ml/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Predictions 2027
predictions_2027 = pd.DataFrame(index=X_predict.index)

rmse_par_candidat = {}
for row in results_rf:
    rmse_par_candidat[row['candidat']] = row['RMSE_mean']

for candidat in candidats_communs:
    nom_court = candidat.split()[-1]
    rf = models_rf[candidat]
    pred = rf.predict(X_predict.values)
    pred = np.clip(pred, 0, 100)  # borner entre 0 et 100
    predictions_2027[f'pred_{nom_court.lower()}'] = pred

# Normaliser a 100% par commune
pred_cols = [c for c in predictions_2027.columns if c.startswith('pred_')]
row_sums = predictions_2027[pred_cols].sum(axis=1)
for col in pred_cols:
    predictions_2027[col] = predictions_2027[col] / row_sums * 100

# Intervalle de confiance : pred +/- 1.96 * RMSE
for candidat in candidats_communs:
    nom_court = candidat.split()[-1].lower()
    rmse = rmse_par_candidat.get(candidat.split()[-1], 3.0)
    predictions_2027[f'ic_inf_{nom_court}'] = np.clip(predictions_2027[f'pred_{nom_court}'] - 1.96 * rmse, 0, 100)
    predictions_2027[f'ic_sup_{nom_court}'] = np.clip(predictions_2027[f'pred_{nom_court}'] + 1.96 * rmse, 0, 100)

print(f"Predictions 2027 : {predictions_2027.shape}")
print(f"\nMoyennes (non ponderees) :")
for col in pred_cols:
    print(f"  {col:25s} : {predictions_2027[col].mean():.2f}%")

In [ ]:
# Visualisation predictions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 1. Barplot predictions nationales (moyenne ponderee par population)
pop_align = df_communes.set_index('id_commune')['population'].reindex(predictions_2027.index).fillna(0)
weighted_means = {}
for col in pred_cols:
    nom = col.replace('pred_', '').upper()
    if pop_align.sum() > 0:
        weighted_means[nom] = np.average(predictions_2027[col], weights=pop_align)
    else:
        weighted_means[nom] = predictions_2027[col].mean()

sorted_cands = sorted(weighted_means.items(), key=lambda x: x[1], reverse=True)
names = [c[0] for c in sorted_cands]
values = [c[1] for c in sorted_cands]

bars = ax1.barh(names, values, color='coral')
for i, v in enumerate(values):
    ax1.text(v + 0.3, i, f'{v:.1f}%', va='center')
ax1.set_xlabel('% voix exprimes predit')
ax1.set_title('Predictions 2027 T1 (moyenne ponderee population)')
ax1.invert_yaxis()

# 2. Scatter predicted vs actual (validation 2022)
all_actual = []
all_predicted = []
for candidat in candidats_communs:
    y_actual = targets[candidat].values.astype(float)
    y_pred = models_rf[candidat].predict(X_train.values)
    all_actual.extend(y_actual)
    all_predicted.extend(y_pred)

ax2.scatter(all_actual, all_predicted, alpha=0.2, s=10, c='steelblue')
lims = [0, max(max(all_actual), max(all_predicted)) + 5]
ax2.plot(lims, lims, 'r--', alpha=0.5, label='y=x (parfait)')
ax2.set_xlabel('% reel (2022 T1)')
ax2.set_ylabel('% predit (modele RF)')
ax2.set_title('Predicted vs Actual (validation sur train set)')
ax2.legend()

plt.tight_layout()
plt.savefig('../docs/figures/ml/predictions_2027.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Sauvegarder en base via ORM - table Prediction
save_session = get_session()

# Supprimer predictions existantes v1.0.0
save_session.query(Prediction).filter(
    Prediction.version_modele == 'v1.0.0',
    Prediction.annee_prediction == 2027
).delete()
save_session.commit()

# Preparer metriques et features
feature_list = list(X_train.columns)
count = 0

for candidat in candidats_communs:
    nom_court = candidat.split()[-1].lower()
    nom_affichage = candidat

    # Metriques du modele
    metrics_row = [r for r in results_rf if r['candidat'] == candidat.split()[-1]][0]
    metriques = {
        'r2': round(metrics_row['R2_mean'], 4),
        'mae': round(metrics_row['MAE_mean'], 4),
        'rmse': round(metrics_row['RMSE_mean'], 4),
        'feature_importance': dict(zip(
            feature_list,
            [round(float(x), 4) for x in models_rf[candidat].feature_importances_]
        ))
    }

    for commune_id in predictions_2027.index:
        pct = float(predictions_2027.loc[commune_id, f'pred_{nom_court}'])
        ic_inf = float(predictions_2027.loc[commune_id, f'ic_inf_{nom_court}'])
        ic_sup = float(predictions_2027.loc[commune_id, f'ic_sup_{nom_court}'])

        prediction = Prediction(
            id_territoire=str(commune_id),
            type_territoire='COMMUNE',
            candidat=nom_affichage,
            parti=None,
            annee_prediction=2027,
            tour=1,
            pourcentage_predit=round(pct, 2),
            intervalle_confiance_inf=round(ic_inf, 2),
            intervalle_confiance_sup=round(ic_sup, 2),
            modele_utilise='RandomForest',
            version_modele='v1.0.0',
            metriques_modele=metriques,
            features_utilisees=feature_list
        )
        save_session.add(prediction)
        count += 1

    # Commit par candidat
    save_session.commit()
    print(f"  {nom_affichage} : {len(predictions_2027)} predictions sauvegardees")

print(f"\nTotal : {count} predictions sauvegardees en base")

# Verification via ORM
nb = save_session.query(func.count()).select_from(Prediction).filter(
    Prediction.version_modele == 'v1.0.0'
).scalar()
print(f"Verification : {nb} lignes dans prediction")

save_session.close()

## Conclusions

### Resultats

- **7 modeles Random Forest** entraines (1 par candidat commun 2017-2022)
- **Random Forest surpasse Linear Regression** sur la majorite des candidats
- **Predictions 2027** generees pour 534 communes x 7 candidats
- **Intervalles de confiance** calcules (pred +/- 1.96 * RMSE)
- **Predictions sauvegardees** en base PostgreSQL (table `prediction`) via ORM

### Features les plus importantes

- Le % de voix du candidat a l'election precedente est la feature dominante
- Le taux de participation est un facteur secondaire significatif
- La population de la commune influence les scores (urbain vs rural)
- Les indicateurs securite ont un impact limite (disponibles uniquement pour Bordeaux)

### Limites

- **2 points temporels** seulement (2017, 2022) : pas de serie temporelle longue
- **Indicateurs securite** disponibles uniquement pour Bordeaux (33063)
- **Hypothese de stabilite** : les candidats 2027 sont supposes identiques a 2017-2022
- **Pas de sondages** integres dans le modele
- **Normalisation a 100%** peut biaiser les predictions individuelles